In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
DATA_PATH = "/content/drive/My Drive/Intro to AI Project/data/sample-10/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple

     |████████████████████████████████| 645kB 13.5MB/s 


In [3]:
import pickle
import json
import os
import numpy as np
from tqdm import *
import gpt_2_simple as gpt2
from datetime import datetime
import tensorflow as tf
import shutil

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [4]:
!nvidia-smi

Thu Dec  5 02:47:08 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
def process_data(i):
    data = pickle.load(open(DATA_PATH + 'recipe1m_train-%d.pkl'%i, 'rb'))
    recipe_file = open(DATA_PATH + "recipe.txt","w")
    for recipe in tqdm(data):
        recipe_file.write("<|startoftext|>")# recipe
        recipe_file.write("<soi>") # ingredients
        recipe_file.write(",".join(recipe["ingredients"]).replace("_"," "))
        recipe_file.write("<eoi>\n")
        recipe_file.write("<sot>") # title
        recipe_file.write(recipe["title"])
        recipe_file.write("<eot>\n")
        recipe_file.write("<soc>-") # content (instructions)
        recipe_file.write("\n-".join(recipe["instructions"]))
        recipe_file.write("<eoc>")
        recipe_file.write("<|endoftext|>\n")
    recipe_file.close()
    # save file to google drive
    file_name = shutil.copyfile(DATA_PATH + "recipe.txt","recipe.txt")

In [0]:
file_name = "recipe.txt"
model_name = '355M'
checkpoint = "/content/drive/My Drive/Intro to AI Project/model/split10-355M/checkpoint"

In [0]:
gpt2.download_gpt2(model_name=model_name)

Fetching checkpoint: 1.05Mit [00:00, 407Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 109Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 573Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:10, 130Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 406Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 94.9Mit/s]                                                
Fetching vocab.bpe: 1.05Mit [00:00, 181Mit/s]                                                       


In [0]:
# run 0
process_data(0)
sess = gpt2.start_tf_sess()
gpt2.finetune(sess,
              dataset=file_name,
              model_name=model_name,
              steps=1000,
              restore_from='fresh',
              checkpoint_dir=checkpoint,
              run_name='run0',
              print_every=1,
              sample_every=200,
              save_every=200
              )


100%|██████████| 63974/63974 [00:00<00:00, 122946.66it/s]


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint models/355M/model.ckpt
INFO:tensorflow:Restoring parameters from models/355M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [01:06<00:00, 66.80s/it]


dataset has 13962469 tokens
Training...
[1 | 11.72] loss=2.27 avg=2.27
[2 | 12.61] loss=2.53 avg=2.40
[3 | 13.50] loss=2.37 avg=2.39
[4 | 14.38] loss=2.29 avg=2.36
[5 | 15.27] loss=2.47 avg=2.38
[6 | 16.16] loss=2.14 avg=2.34
[7 | 17.05] loss=2.09 avg=2.31
[8 | 17.93] loss=1.99 avg=2.26
[9 | 18.81] loss=2.52 avg=2.29
[10 | 19.70] loss=1.86 avg=2.25
[11 | 20.58] loss=2.09 avg=2.23
[12 | 21.48] loss=2.17 avg=2.23
[13 | 22.36] loss=1.84 avg=2.20
[14 | 23.25] loss=2.07 avg=2.19
[15 | 24.13] loss=2.15 avg=2.18
[16 | 25.02] loss=1.72 avg=2.15
[17 | 25.90] loss=1.95 avg=2.14
[18 | 26.79] loss=1.86 avg=2.12
[19 | 27.67] loss=2.59 avg=2.15
[20 | 28.56] loss=1.87 avg=2.13
[21 | 29.44] loss=2.14 avg=2.13
[22 | 30.33] loss=1.84 avg=2.12
[23 | 31.21] loss=1.90 avg=2.11
[24 | 32.10] loss=1.79 avg=2.09
[25 | 32.99] loss=1.84 avg=2.08
[26 | 33.87] loss=1.88 avg=2.07
[27 | 34.77] loss=1.87 avg=2.06
[28 | 35.65] loss=1.80 avg=2.05
[29 | 36.54] loss=2.01 avg=2.05
[30 | 37.42] loss=1.81 avg=2.04
[31 | 38.

In [0]:
i = 1
process_data(i)
tf.reset_default_graph()
sess = gpt2.start_tf_sess()
gpt2.finetune(sess,
              dataset=file_name,
              model_name=model_name,
              steps=1000,
              restore_from=checkpoint+"/"+'run%d'%(i-1),
              checkpoint_dir=checkpoint,
              run_name='run%d'%i,
              print_every=1,
              sample_every=200,
              save_every=200
              )

100%|██████████| 63974/63974 [00:00<00:00, 143803.42it/s]


Loading checkpoint /content/drive/My Drive/Intro to AI Project/model/split10-355M/checkpoint/run0/model-1000
INFO:tensorflow:Restoring parameters from /content/drive/My Drive/Intro to AI Project/model/split10-355M/checkpoint/run0/model-1000


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [01:07<00:00, 67.45s/it]


dataset has 13971134 tokens
Training...
[1 | 10.49] loss=1.53 avg=1.53
[2 | 11.38] loss=1.54 avg=1.53
[3 | 12.26] loss=1.73 avg=1.60
[4 | 13.15] loss=1.62 avg=1.61
[5 | 14.03] loss=1.80 avg=1.65
[6 | 14.91] loss=1.81 avg=1.67
[7 | 15.79] loss=1.80 avg=1.69
[8 | 16.69] loss=1.80 avg=1.71
[9 | 17.57] loss=1.72 avg=1.71
[10 | 18.46] loss=1.44 avg=1.68
[11 | 19.34] loss=1.82 avg=1.69
[12 | 20.23] loss=1.85 avg=1.71
[13 | 21.11] loss=1.80 avg=1.71
[14 | 21.99] loss=1.94 avg=1.73
[15 | 22.88] loss=1.79 avg=1.74
[16 | 23.77] loss=1.67 avg=1.73
[17 | 24.65] loss=1.41 avg=1.71
[18 | 25.53] loss=1.44 avg=1.70
[19 | 26.43] loss=1.72 avg=1.70
[20 | 27.31] loss=1.55 avg=1.69
[21 | 28.20] loss=1.71 avg=1.69
[22 | 29.08] loss=1.59 avg=1.68
[23 | 29.97] loss=1.83 avg=1.69
[24 | 30.85] loss=1.78 avg=1.70
[25 | 31.74] loss=1.85 avg=1.70
[26 | 32.62] loss=1.71 avg=1.70
[27 | 33.51] loss=1.37 avg=1.69
[28 | 34.41] loss=1.88 avg=1.70
[29 | 35.29] loss=1.55 avg=1.69
[30 | 36.18] loss=1.72 avg=1.69
[31 | 37.

In [0]:
i = 2
process_data(i)
tf.reset_default_graph()
sess = gpt2.start_tf_sess()
gpt2.finetune(sess,
              dataset=file_name,
              model_name=model_name,
              steps=1000,
              restore_from=checkpoint+"/"+'run%d'%(i-1),
              checkpoint_dir=checkpoint,
              run_name='run%d'%i,
              print_every=1,
              sample_every=200,
              save_every=200
              )

100%|██████████| 63974/63974 [00:00<00:00, 138527.30it/s]


Loading checkpoint /content/drive/My Drive/Intro to AI Project/model/split10-355M/checkpoint/run1/model-1000
INFO:tensorflow:Restoring parameters from /content/drive/My Drive/Intro to AI Project/model/split10-355M/checkpoint/run1/model-1000


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [01:04<00:00, 64.50s/it]


dataset has 13979524 tokens
Training...
[1 | 11.52] loss=1.47 avg=1.47
[2 | 12.42] loss=1.56 avg=1.51
[3 | 13.31] loss=1.87 avg=1.63
[4 | 14.20] loss=1.96 avg=1.72
[5 | 15.08] loss=1.49 avg=1.67
[6 | 15.97] loss=1.99 avg=1.72
[7 | 16.85] loss=1.45 avg=1.68
[8 | 17.74] loss=1.63 avg=1.68
[9 | 18.62] loss=1.48 avg=1.65
[10 | 19.52] loss=1.65 avg=1.65
[11 | 20.41] loss=1.71 avg=1.66
[12 | 21.31] loss=1.48 avg=1.64
[13 | 22.20] loss=1.55 avg=1.64
[14 | 23.09] loss=1.49 avg=1.63
[15 | 23.98] loss=1.39 avg=1.61
[16 | 24.87] loss=1.69 avg=1.61
[17 | 25.76] loss=1.58 avg=1.61
[18 | 26.66] loss=1.58 avg=1.61
[19 | 27.55] loss=1.72 avg=1.62
[20 | 28.43] loss=1.59 avg=1.62
[21 | 29.32] loss=1.51 avg=1.61
[22 | 30.20] loss=1.65 avg=1.61
[23 | 31.09] loss=1.56 avg=1.61
[24 | 31.98] loss=1.47 avg=1.60
[25 | 32.86] loss=1.48 avg=1.60
[26 | 33.75] loss=1.65 avg=1.60
[27 | 34.64] loss=1.44 avg=1.59
[28 | 35.54] loss=1.60 avg=1.59
[29 | 36.42] loss=1.52 avg=1.59
[30 | 37.31] loss=1.46 avg=1.58
[31 | 38.

In [0]:
i = 3
process_data(i)
tf.reset_default_graph()
sess = gpt2.start_tf_sess()
gpt2.finetune(sess,
              dataset=file_name,
              model_name=model_name,
              steps=1000,
              restore_from=checkpoint+"/"+'run%d'%(i-1),
              checkpoint_dir=checkpoint,
              run_name='run%d'%i,
              print_every=1,
              sample_every=200,
              save_every=200
              )

100%|██████████| 63974/63974 [00:00<00:00, 127145.53it/s]


Loading checkpoint /content/drive/My Drive/Intro to AI Project/model/split10-355M/checkpoint/run2/model-1000
INFO:tensorflow:Restoring parameters from /content/drive/My Drive/Intro to AI Project/model/split10-355M/checkpoint/run2/model-1000


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [01:04<00:00, 64.70s/it]


dataset has 14014371 tokens
Training...
[1 | 10.69] loss=1.59 avg=1.59
[2 | 11.58] loss=1.72 avg=1.65
[3 | 12.46] loss=1.57 avg=1.62
[4 | 13.34] loss=1.57 avg=1.61
[5 | 14.23] loss=1.31 avg=1.55
[6 | 15.11] loss=1.65 avg=1.57
[7 | 16.00] loss=1.73 avg=1.59
[8 | 16.89] loss=1.73 avg=1.61
[9 | 17.77] loss=1.64 avg=1.61
[10 | 18.66] loss=1.57 avg=1.61
[11 | 19.55] loss=1.45 avg=1.59
[12 | 20.44] loss=1.33 avg=1.57
[13 | 21.33] loss=1.77 avg=1.59
[14 | 22.22] loss=1.52 avg=1.58
[15 | 23.10] loss=1.49 avg=1.57
[16 | 23.98] loss=1.89 avg=1.60
[17 | 24.89] loss=1.88 avg=1.61
[18 | 25.77] loss=1.54 avg=1.61
[19 | 26.66] loss=1.42 avg=1.60
[20 | 27.54] loss=1.34 avg=1.58
[21 | 28.43] loss=1.84 avg=1.60
[22 | 29.32] loss=1.47 avg=1.59
[23 | 30.20] loss=1.87 avg=1.60
[24 | 31.09] loss=1.50 avg=1.60
[25 | 31.97] loss=1.52 avg=1.60
[26 | 32.86] loss=1.35 avg=1.58
[27 | 33.74] loss=1.74 avg=1.59
[28 | 34.63] loss=1.60 avg=1.59
[29 | 35.51] loss=1.43 avg=1.59
[30 | 36.39] loss=1.56 avg=1.58
[31 | 37.

In [0]:
i = 4
process_data(i)
tf.reset_default_graph()
sess = gpt2.start_tf_sess()
gpt2.finetune(sess,
              dataset=file_name,
              model_name=model_name,
              steps=1000,
              restore_from=checkpoint+"/"+'run%d'%(i-1),
              checkpoint_dir=checkpoint,
              run_name='run%d'%i,
              print_every=1,
              sample_every=200,
              save_every=200
              )

100%|██████████| 63974/63974 [00:00<00:00, 140356.65it/s]


Loading checkpoint /content/drive/My Drive/Intro to AI Project/model/split10-355M/checkpoint/run3/model-1000
INFO:tensorflow:Restoring parameters from /content/drive/My Drive/Intro to AI Project/model/split10-355M/checkpoint/run3/model-1000


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [01:05<00:00, 65.56s/it]


dataset has 13984744 tokens
Training...
[1 | 11.17] loss=1.30 avg=1.30
[2 | 12.07] loss=1.36 avg=1.33
[3 | 12.96] loss=1.63 avg=1.43
[4 | 13.85] loss=1.61 avg=1.48
[5 | 14.74] loss=1.67 avg=1.52
[6 | 15.62] loss=1.70 avg=1.55
[7 | 16.50] loss=1.48 avg=1.54
[8 | 17.39] loss=1.52 avg=1.53
[9 | 18.28] loss=1.73 avg=1.56
[10 | 19.17] loss=1.38 avg=1.54
[11 | 20.05] loss=1.21 avg=1.51
[12 | 20.94] loss=1.60 avg=1.52
[13 | 21.82] loss=1.48 avg=1.51
[14 | 22.71] loss=1.69 avg=1.53
[15 | 23.59] loss=1.46 avg=1.52
[16 | 24.48] loss=1.55 avg=1.52
[17 | 25.36] loss=2.07 avg=1.56
[18 | 26.26] loss=1.43 avg=1.55
[19 | 27.15] loss=1.40 avg=1.54
[20 | 28.03] loss=1.79 avg=1.55
[21 | 28.92] loss=1.42 avg=1.55
[22 | 29.80] loss=1.48 avg=1.54
[23 | 30.69] loss=1.48 avg=1.54
[24 | 31.59] loss=1.52 avg=1.54
[25 | 32.48] loss=1.56 avg=1.54
[26 | 33.37] loss=1.84 avg=1.55
[27 | 34.26] loss=1.48 avg=1.55
[28 | 35.14] loss=1.29 avg=1.54
[29 | 36.02] loss=1.42 avg=1.54
[30 | 36.91] loss=1.73 avg=1.54
[31 | 37.

In [0]:
i = 5
process_data(i)
tf.reset_default_graph()
sess = gpt2.start_tf_sess()
gpt2.finetune(sess,
              dataset=file_name,
              model_name=model_name,
              steps=1000,
              restore_from=checkpoint+"/"+'run%d'%(i-1),
              checkpoint_dir=checkpoint,
              run_name='run%d'%i,
              print_every=1,
              sample_every=200,
              save_every=200
              )

100%|██████████| 63974/63974 [00:00<00:00, 158951.16it/s]


Loading checkpoint /content/drive/My Drive/Intro to AI Project/model/split10-355M/checkpoint/run4/model-1000
INFO:tensorflow:Restoring parameters from /content/drive/My Drive/Intro to AI Project/model/split10-355M/checkpoint/run4/model-1000


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [01:01<00:00, 61.72s/it]


dataset has 14012473 tokens
Training...
[1 | 11.10] loss=1.32 avg=1.32
[2 | 12.00] loss=1.65 avg=1.48
[3 | 12.89] loss=1.27 avg=1.41
[4 | 13.78] loss=1.34 avg=1.39
[5 | 14.66] loss=1.97 avg=1.51
[6 | 15.55] loss=1.53 avg=1.51
[7 | 16.43] loss=1.65 avg=1.53
[8 | 17.32] loss=1.60 avg=1.54
[9 | 18.21] loss=1.45 avg=1.53
[10 | 19.11] loss=1.84 avg=1.56
[11 | 20.00] loss=1.49 avg=1.56
[12 | 20.89] loss=1.55 avg=1.56
[13 | 21.77] loss=1.33 avg=1.54
[14 | 22.66] loss=2.22 avg=1.59
[15 | 23.54] loss=1.54 avg=1.59
[16 | 24.43] loss=1.58 avg=1.59
[17 | 25.31] loss=1.60 avg=1.59
[18 | 26.20] loss=1.45 avg=1.58
[19 | 27.08] loss=1.56 avg=1.58
[20 | 27.97] loss=1.31 avg=1.56
[21 | 28.86] loss=1.97 avg=1.58
[22 | 29.74] loss=1.90 avg=1.60
[23 | 30.63] loss=1.91 avg=1.62
[24 | 31.52] loss=1.21 avg=1.60
[25 | 32.40] loss=1.41 avg=1.59
[26 | 33.29] loss=1.27 avg=1.57
[27 | 34.17] loss=1.64 avg=1.58
[28 | 35.06] loss=1.46 avg=1.57
[29 | 35.95] loss=1.49 avg=1.57
[30 | 36.84] loss=1.51 avg=1.57
[31 | 37.

In [0]:
i = 6
process_data(i)
tf.reset_default_graph()
sess = gpt2.start_tf_sess()
gpt2.finetune(sess,
              dataset=file_name,
              model_name=model_name,
              steps=1000,
              restore_from=checkpoint+"/"+'run%d'%(i-1),
              checkpoint_dir=checkpoint,
              run_name='run%d'%i,
              print_every=1,
              sample_every=200,
              save_every=200
              )

100%|██████████| 63974/63974 [00:00<00:00, 189616.70it/s]


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint /content/drive/My Drive/Intro to AI Project/model/split10-355M/checkpoint/run5/model-1000
INFO:tensorflow:Restoring parameters from /content/drive/My Drive/Intro to AI Project/model/split10-355M/checkpoint/run5/model-1000


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:57<00:00, 57.44s/it]


dataset has 13965454 tokens
Training...
[1 | 10.50] loss=1.18 avg=1.18
[2 | 11.39] loss=1.53 avg=1.36
[3 | 12.27] loss=1.80 avg=1.51
[4 | 13.16] loss=1.37 avg=1.47
[5 | 14.05] loss=1.57 avg=1.49
[6 | 14.93] loss=1.44 avg=1.48
[7 | 15.81] loss=1.56 avg=1.49
[8 | 16.70] loss=1.33 avg=1.47
[9 | 17.58] loss=1.48 avg=1.47
[10 | 18.46] loss=1.30 avg=1.46
[11 | 19.34] loss=1.34 avg=1.44
[12 | 20.23] loss=1.77 avg=1.47
[13 | 21.11] loss=1.33 avg=1.46
[14 | 21.99] loss=1.67 avg=1.48
[15 | 22.87] loss=1.78 avg=1.50
[16 | 23.77] loss=1.77 avg=1.52
[17 | 24.65] loss=1.30 avg=1.50
[18 | 25.53] loss=1.28 avg=1.49
[19 | 26.42] loss=1.51 avg=1.49
[20 | 27.30] loss=1.34 avg=1.48
[21 | 28.18] loss=1.37 avg=1.48
[22 | 29.06] loss=1.72 avg=1.49
[23 | 29.95] loss=1.85 avg=1.51
[24 | 30.83] loss=1.26 avg=1.49
[25 | 31.71] loss=1.59 avg=1.50
[26 | 32.60] loss=1.42 avg=1.50
[27 | 33.48] loss=1.59 avg=1.50
[28 | 34.36] loss=1.54 avg=1.50
[29 | 35.25] loss=1.28 avg=1.49
[30 | 36.14] loss=1.48 avg=1.49
[31 | 37.

In [0]:
i = 7
process_data(i)
tf.reset_default_graph()
sess = gpt2.start_tf_sess()
gpt2.finetune(sess,
              dataset=file_name,
              model_name=model_name,
              steps=1000,
              restore_from=checkpoint+"/"+'run%d'%(i-1),
              checkpoint_dir=checkpoint,
              run_name='run%d'%i,
              print_every=1,
              sample_every=200,
              save_every=200
              )

100%|██████████| 63974/63974 [00:00<00:00, 195245.31it/s]


Loading checkpoint /content/drive/My Drive/Intro to AI Project/model/split10-355M/checkpoint/run6/model-1000
INFO:tensorflow:Restoring parameters from /content/drive/My Drive/Intro to AI Project/model/split10-355M/checkpoint/run6/model-1000


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:52<00:00, 52.99s/it]


dataset has 13944705 tokens
Training...
[1 | 9.62] loss=1.40 avg=1.40
[2 | 10.51] loss=1.59 avg=1.49
[3 | 11.39] loss=1.41 avg=1.47
[4 | 12.28] loss=1.85 avg=1.56
[5 | 13.16] loss=1.36 avg=1.52
[6 | 14.04] loss=1.66 avg=1.55
[7 | 14.93] loss=1.36 avg=1.52
[8 | 15.81] loss=1.54 avg=1.52
[9 | 16.70] loss=1.92 avg=1.57
[10 | 17.58] loss=1.76 avg=1.59
[11 | 18.46] loss=1.63 avg=1.59
[12 | 19.34] loss=1.56 avg=1.59
[13 | 20.22] loss=1.63 avg=1.59
[14 | 21.11] loss=1.31 avg=1.57
[15 | 21.99] loss=1.40 avg=1.56
[16 | 22.87] loss=1.49 avg=1.55
[17 | 23.76] loss=1.44 avg=1.55
[18 | 24.65] loss=1.14 avg=1.52
[19 | 25.53] loss=1.44 avg=1.52
[20 | 26.41] loss=1.16 avg=1.50
[21 | 27.30] loss=1.44 avg=1.49
[22 | 28.18] loss=1.71 avg=1.50
[23 | 29.06] loss=1.42 avg=1.50
[24 | 29.94] loss=1.69 avg=1.51
[25 | 30.83] loss=1.36 avg=1.50
[26 | 31.71] loss=1.59 avg=1.51
[27 | 32.60] loss=1.78 avg=1.52
[28 | 33.48] loss=1.35 avg=1.51
[29 | 34.36] loss=1.59 avg=1.51
[30 | 35.24] loss=1.36 avg=1.51
[31 | 36.1

In [0]:
i = 8
process_data(i)
tf.reset_default_graph()
sess = gpt2.start_tf_sess()
gpt2.finetune(sess,
              dataset=file_name,
              model_name=model_name,
              steps=1000,
              restore_from=checkpoint+"/"+'run%d'%(i-1),
              checkpoint_dir=checkpoint,
              run_name='run%d'%i,
              print_every=1,
              sample_every=200,
              save_every=200
              )

100%|██████████| 63974/63974 [00:00<00:00, 121770.39it/s]


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint /content/drive/My Drive/Intro to AI Project/model/split10-355M/checkpoint/run7/model-1000
INFO:tensorflow:Restoring parameters from /content/drive/My Drive/Intro to AI Project/model/split10-355M/checkpoint/run7/model-1000


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [01:06<00:00, 66.35s/it]


dataset has 13996102 tokens
Training...
[1 | 14.01] loss=1.48 avg=1.48
[2 | 17.01] loss=1.41 avg=1.45
[3 | 20.00] loss=1.37 avg=1.42
[4 | 23.00] loss=1.46 avg=1.43
[5 | 26.00] loss=1.40 avg=1.43
[6 | 29.01] loss=1.31 avg=1.41
[7 | 32.00] loss=1.57 avg=1.43
[8 | 35.02] loss=1.55 avg=1.45
[9 | 38.03] loss=1.48 avg=1.45
[10 | 41.05] loss=1.57 avg=1.46
[11 | 44.06] loss=1.59 avg=1.48
[12 | 47.08] loss=1.28 avg=1.46
[13 | 50.11] loss=1.52 avg=1.46
[14 | 53.14] loss=1.42 avg=1.46
[15 | 56.16] loss=1.34 avg=1.45
[16 | 59.20] loss=1.65 avg=1.46
[17 | 62.24] loss=1.65 avg=1.48
[18 | 65.28] loss=1.60 avg=1.48
[19 | 68.33] loss=1.48 avg=1.48
[20 | 71.37] loss=1.33 avg=1.48
[21 | 74.43] loss=1.33 avg=1.47
[22 | 77.47] loss=1.37 avg=1.46
[23 | 80.52] loss=1.37 avg=1.46
[24 | 83.58] loss=1.36 avg=1.45
[25 | 86.63] loss=1.81 avg=1.47
[26 | 89.68] loss=1.55 avg=1.47
[27 | 92.72] loss=1.16 avg=1.46
[28 | 95.77] loss=1.26 avg=1.45
[29 | 98.81] loss=1.48 avg=1.45
[30 | 101.86] loss=1.39 avg=1.45
[31 | 10

In [0]:
i = 9
process_data(i)
tf.reset_default_graph()
sess = gpt2.start_tf_sess()
gpt2.finetune(sess,
              dataset=file_name,
              model_name=model_name,
              steps=1000,
              restore_from=checkpoint+"/"+'run%d'%(i-1),
              checkpoint_dir=checkpoint,
              run_name='run%d'%i,
              print_every=1,
              sample_every=200,
              save_every=200
              )

100%|██████████| 63974/63974 [00:00<00:00, 152496.65it/s]


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint /content/drive/My Drive/Intro to AI Project/model/split10-355M/checkpoint/run8/model-1000
INFO:tensorflow:Restoring parameters from /content/drive/My Drive/Intro to AI Project/model/split10-355M/checkpoint/run8/model-1000


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [01:05<00:00, 65.17s/it]


dataset has 13959912 tokens
Training...
[1 | 13.46] loss=1.59 avg=1.59
[2 | 16.62] loss=1.51 avg=1.55
[3 | 19.77] loss=1.46 avg=1.52
[4 | 22.92] loss=1.51 avg=1.52
[5 | 26.07] loss=1.27 avg=1.47
[6 | 29.23] loss=1.39 avg=1.46
[7 | 32.37] loss=1.65 avg=1.48
[8 | 35.52] loss=1.14 avg=1.44
[9 | 38.65] loss=1.44 avg=1.44
[10 | 41.79] loss=1.85 avg=1.48
[11 | 44.92] loss=1.48 avg=1.48
[12 | 48.05] loss=1.34 avg=1.47
[13 | 51.18] loss=1.44 avg=1.47
[14 | 54.31] loss=1.67 avg=1.48
[15 | 57.45] loss=1.36 avg=1.47
[16 | 60.58] loss=1.48 avg=1.47
[17 | 63.71] loss=0.99 avg=1.44
[18 | 66.84] loss=1.07 avg=1.42
[19 | 69.98] loss=1.16 avg=1.41
[20 | 73.10] loss=1.38 avg=1.40
[21 | 76.23] loss=1.43 avg=1.41
[22 | 79.35] loss=1.55 avg=1.41
[23 | 82.47] loss=1.51 avg=1.42
[24 | 85.60] loss=1.65 avg=1.43
[25 | 88.70] loss=1.73 avg=1.44
[26 | 91.82] loss=1.51 avg=1.44
[27 | 94.94] loss=1.45 avg=1.44
[28 | 98.08] loss=2.02 avg=1.47
[29 | 101.21] loss=1.52 avg=1.47
[30 | 104.33] loss=1.06 avg=1.45
[31 | 1

In [7]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run9',checkpoint_dir=checkpoint)

Loading checkpoint /content/drive/My Drive/Intro to AI Project/model/split10-355M/checkpoint/run9/model-1000
INFO:tensorflow:Restoring parameters from /content/drive/My Drive/Intro to AI Project/model/split10-355M/checkpoint/run9/model-1000


In [8]:
# val 999
gpt2.generate(sess,
              run_name='run9',
              length=1024,
              temperature=1.0,
              prefix="<|startoftext|><soi>all purpose flour,whole wheat flour,baking soda,salt,quick cooking oat,all bran cereal,raisin bran cereal,grape nut cereal,raisin,unsalted butter,buttermilk,granulated sugar,brown sugar,egg<eoi>",
              truncate="<|endoftext|>",
              checkpoint_dir=checkpoint,
              nsamples=5,
              batch_size=5
              )

<|startoftext|><soi>all purpose flour,whole wheat flour,baking soda,salt,quick cooking oat,all bran cereal,raisin bran cereal,grape nut cereal,raisin,unsalted butter,buttermilk,granulated sugar,brown sugar,egg<eoi>
<sot>bran bran bran muffins<eot>
<soc>-warm oven to 220 degrees.
-set oven rack to middle position and position 1 rack in centre of oven.
-in 1 large bowl, add flour, wheat flour, baking soda, salt and oats.
-in 1 food processor, pulse oats, bran, raisins, oats rye dry cereal, grapes, raisins, grapes, raisins, raisins, raisins, grapes, raisins, grapes, raisins, grapes, grapes, raisins, dough will be crumbly, and will break up .
-meanwhile, in small bowl, sift together flour, bran mixture, raisins, butter, buttermilk, sugar, brown sugar and eggs.
-stir in oats mixture and stir gently until combined.
-fill greased or lined muffin tins three quarters full with the batter.
-bake at 180 degrees for 45 60 min or until golden brown.
-**if the bran bran pops, try lightly sprinkling 

In [0]:
#val 104
gpt2.generate(sess,
              run_name='run0',
              length=250,
              temperature=1.0,
              prefix="<|startoftext|><soi>egg,light brown sugar,buttermilk,water,brewed coffee,all purpose white flour,baking powder,baking soda,salt,ground clove,ground cinnamon,ground ginger,ground nutmeg,butter<eoi>",
              truncate="<|endoftext|>",
              checkpoint_dir=checkpoint,
              nsamples=5,
              batch_size=5
              )

<|startoftext|><soi>egg,light brown sugar,buttermilk,water,brewed coffee,all purpose white flour,baking powder,baking soda,salt,ground clove,ground cinnamon,ground ginger,ground nutmeg,butter<eoi>
<sot>energy made coffee coffee cake<eot>
<soc>-heat oven to 350 degrees f.
-grease and flour 3 24" greased loaf pans or two 9in/2 in springform pans.
-grease bottom of each pan. set aside.
-put dates into 1 saucepan. add rest of ingredients, except butter.
-bring to 1 boil, stirring often but do not let it boil.
-sift together flour, baking powder, baking soda all at once and salt.
-pour sugar mixture and slowly add dates. mix well.
-add flour mixture to dates or alternately with butter, stirring to combine.
-pour into pan and bake for 35 40 minutes, or until 1 toothpick comes out clean.
-cool in pan 10 minutes, then insert wooden finger in centre of cake.<eoc>
<|startoftext|><soi>egg,light brown sugar,buttermilk,water,brewed coffee,all purpose white flour,baking powder,baking soda,salt,groun

In [0]:
#val 16
gpt2.generate(sess,
              run_name='run0',
              length=1024,
              temperature=0.8,
              prefix="<|startoftext|><soi>chicken wing,cilantro,garlic clove,jalapeno,fish sauce,fresh lime juice,unsalted butter,flour,canola oil,kosher salt,black pepper<eoi>",
              truncate="<|endoftext|>",
              checkpoint_dir=checkpoint,
              nsamples=5,
              batch_size=5
              )

<|startoftext|><soi>chicken wing,cilantro,garlic clove,jalapeno,fish sauce,fresh lime juice,unsalted butter,flour,canola oil,kosher salt,black pepper<eoi>
<sot>jalapeno cilantro chicken wings<eot>
<soc>-combine cilantro, garlic, jalapeno, fish sauce, lime juice, wok or 1 large skillet, and butter in 1 large bowl and stir to combine.
-cook wings over high heat, turning once, about 4 minutes.
-remove wings to bowl and let stand, covered, until cool enough to handle, about 15 minutes.
-transfer wings to 1 large bowl and stir until well coated with cilantro.
-wipe out bowl and immediately refrigerate.
-cook wings in the same manner as you would 1 chicken breast, adding additional oil to drippings if necessary.
-transfer wings to 1 serving platter or 1 large, shallow baking dish.
-return remaining oil to skillet and heat.
-add the chicken pieces and stir to coat.
-cook, stirring occasionally, until the skin is crispy, about 4 minutes.
-transfer with 1 slotted spoon to 1 platter or baking sh